# Segmenting and Clustering Neighborhoods in Toronto

---

## Summary

In this notebook, I have segmented and clustered the neighborhoods in Toronto, Canada, based on their nearby venues. In order to do this, I have first scraped the postal codes, boroughs, and neighborhoods of Toronto from a table from Wikipedia and formatted these into a DataFrame. After that, I have downloaded the geographical coordinates from Coursera, and merged these with the existing DataFrame. These coordinates made me able to perform API calls to Foursquare, which results in a DataFrame of each venue in each neighborhood, including their geographical coordinates and category.

Using these venues and **K-Means Clustering**, the neighborhoods were splitted into **four clusters**: 

* The **first cluster** contains 67 neighborhoods which, compared to the other clusters, maily include a park, coffee shops, and pizza places. 
* The **second cluster** contains 12 neighborhoods, and all these neighborhoods include a coffee shop and a cafe. Also, this cluster contain seven vegetarian restaurants and six art galleries, which are not in the top 10 of the other clusters. 
* The **third cluster** only contains one neighborhood. However, this neighborhood includes a wide variation of venues, from a bakery and a grocery store to a spa and a pub.
* The **last cluster** contains 18 neighborhoods, which all contain a coffee shop. However, these neighborhoods have a wide variation of places to have a dinner. For example, this cluster include in total 15 restaurants, 12 sandwich places, 10 sushi restaurants, and 8 Italian restaurant.

---

## Getting PostalCode, Borough, and Neighborhoods

First, the required data is scraped from Wikipedia using requests and BeautifulSoup. After that, a for loop is used to append each row data into a list. This list is used to remove the "not assigned" neighborhoods and format the values. Finally, this list is converted to a two-dimensional NumPy, in order to split the columns from the values, and to create a DataFrame.

In [1]:
import requests
from bs4 import BeautifulSoup

# 1. Getting the webpage content of Wikipedia
wikipedia = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").content

# 2. Converting the HTML content to a BeautifulSoup object
wikipedia_soup = BeautifulSoup(wikipedia, "html.parser")

# 3. Assigning the table content from Wikipedia to variable
neighborhoods = wikipedia_soup.table

neighborhoods

<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea

In [2]:
# 4. Converting the tags to lists
neighborhoods_lists = []
for tag in neighborhoods.find_all("tr"):
    temp_list = []
    temp_split = tag.text.split("\n")
    for i in range(len(temp_split)):
        if i in [1, 3, 5]:
            temp_list.append(temp_split[i])
    neighborhoods_lists.append(temp_list)

neighborhoods_lists

[['Postal code', 'Borough', 'Neighborhood'],
 ['M1A', 'Not assigned', ''],
 ['M2A', 'Not assigned', ''],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park / Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor / Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park / Ontario Provincial Government"],
 ['M8A', 'Not assigned', ''],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Malvern / Rouge'],
 ['M2B', 'Not assigned', ''],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill / Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District / Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', ''],
 ['M8B', 'Not assigned', ''],
 ['M9B',
  'Etobicoke',
  'West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill / Port Union / Highland Creek'],
 ['M2C', 'Not assigned', ''],
 ['M3C', 'No

In [3]:
import re

# 5. Removing the "not assigned" boroughs
neighborhoods_rows = []
for lst in neighborhoods_lists:
    if "Not assigned" in lst[1]:
        continue
    else:
        lst[2] = re.sub(" /", ",", lst[2])
        neighborhoods_rows.append(lst)

neighborhoods_rows

[['Postal code', 'Borough', 'Neighborhood'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M3C', 'North York', 'Don Mills'],
 ['M4C', 'East York', 'Woodbine Heights'],
 ['M5C', 'Downtown Toronto', 'St. James Town'],
 ['M6C', 'York', 'Humewood-Cedarvale'],
 ['M9C',
  'Etobicoke',
  'Eringate, Bloordale Gardens, Old Burnha

In [4]:
import numpy as np
import pandas as pd

# 6. Converting the lists into a NumPy array and separates the columns from the values
neighborhoods_rows = np.array(neighborhoods_rows)
neighborhoods_columns = neighborhoods_rows[0,:]
neighborhoods_values = neighborhoods_rows[1:,:]

# 7. Creating a DataFrame of the neighborhoods in Toronto
neighborhoods_df = pd.DataFrame(
    neighborhoods_values,
    columns = neighborhoods_columns
).rename(columns={"Postal code": "PostalCode"}).sort_values(by=["PostalCode"])

# 8. Printing the number of rows in the DataFrame
print("This DataFrame contains {} rows.".format(neighborhoods_df.shape[0]))

neighborhoods_df

This DataFrame contains 103 rows.


,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
...,...,...,...
64,M9N,York,Weston
70,M9P,Etobicoke,Westmount
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


---

## Getting Latitude and Longitude Coordinates

In order to make API calls to Foursquare, the georaphical coordinate of each neighborhood is required. Therefore, the CSV file from Coursera is used and merged with the existing DataFrame. This results in the DataFrame below.

In [5]:
# 1. Loading the coordinates csv into notebook
coordinates_df = pd.read_csv("Files/Geospatial_Coordinates.csv").rename(columns={"Postal Code": "PostalCode"})

# 2. Merging neighborhoods_df with coordinates_df
neighborhoods_df = pd.merge(neighborhoods_df, coordinates_df)

neighborhoods_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


---

## Exploring the Neighborhood (Malvern, Rouge)

In order to explore every neighborhood in Toronto, I have first explored one neighborhood: Malvern, Rouge. In the code below, I have made the first API call with the geographical coordinate of Malvern, Rouge. This results in a JSON file, including the nearby venues in the neighborhood. These data is converted in DataFrame and formatted in such a way that it is readable.

In [6]:
from pandas.io.json import json_normalize

# 1. Defining Foursquare credentials and version
CLIENT_ID = "ITYEMLDDKSZTIRSHWI2SXOPCGFJ3AWQY5JKF3PZVKD5EJLIP"
CLIENT_SECRET = "MFSJZJIIEGJPW2H1TLX1ZEKPACYPWS3KRIOK3HNO3ARJIBQM"
VERSION = "20200504"
radius = 500
limit = 100

# 2. Setting the neighborhood's coordinates
neighborhood_name = neighborhoods_df.loc[0, "Neighborhood"]
neighborhood_latitude = neighborhoods_df.loc[0, "Latitude"]
neighborhood_longitude = neighborhoods_df.loc[0, "Longitude"]

# 3. Creating URL and sends the GET request
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    limit
)
results = requests.get(url).json()

results

{'meta': {'code': 200, 'requestId': '5eb170eff7706a001b14765e'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [7]:
# 4. Getting the items in JSON
venues = results["response"]["groups"][0]["items"]

# 5. Converting JSON into a DataFrame
venues_df = json_normalize(venues)

# 6. Filtering the columns
filtered_columns = ["venue.name", "venue.categories", "venue.location.lat", "venue.location.lng"]
venues_df = venues_df.loc[:, filtered_columns]

# 7. Returning the value of the name key in venue.categories
venues_df["category"] = venues_df["venue.categories"].apply(lambda x: x[0]["name"])

# 8. Renaming the columns
venues_df.columns = [x.split(".")[-1] for x in venues_df.columns]

# 9. Printing the number of venues returned by Foursquare
print("{} venues were returned by Foursquare.".format(venues_df.shape[0]))

# 10. Adding the neighborhood to each row
venues_df["neighborhood"] = neighborhood_name

# 11. Setting neighborhood as first column
venues_df = venues_df[["neighborhood", "name", "category", "lat", "lng"]]

venues_df

2 venues were returned by Foursquare.


,neighborhood,name,category,lat,lng
0,"Malvern, Rouge",Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,"Malvern, Rouge",Interprovincial Group,Print Shop,43.805630,-79.200378


---

## Exploring All Neighborhoods in Toronto

In the code below, API calls are done for each neighborhood in the first DataFrame. These JSON files are appended to a list, including the neigborhood of each venue. Just like the code above, these JSON files are converted and appended to a single DataFrame and formatted in such a way that the neighborhood, category and geographical coordinates of each venue are shown. 

In [8]:
# 1. Assinging necessary column values to variables
neighborhood_values = neighborhoods_df["Neighborhood"].values
latitude_values = neighborhoods_df["Latitude"].values
longitude_values = neighborhoods_df["Longitude"].values

# 2. Sends the GET results for each neighborhood in the DataFrame and appends its JSON in all_venues
all_results = []
all_venues_neighborhoods = []
for hood, lat, lng in zip(neighborhood_values, latitude_values, longitude_values):
    
    # 2.1. Generates the URL
    temp_url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit{}".format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)
    
    # 2.2. Sends the GET request
    temp_results = requests.get(temp_url).json()["response"]["groups"][0]["items"]
    
    # 2.3. Appends the results to all_venues
    all_results.append(temp_results)
    
    # 2.4. Generates a temporary DataFrame in order to get the number of rows
    temp_df = json_normalize(temp_results)
    
    # 2.5. Appends the neighborhood of each row in the temporary DataFrame
    for i in range(temp_df.shape[0]):
        all_venues_neighborhoods.append(hood)

print("Number of results: {}}".format(len(all_venues)))
print("Number of venues: {}".format(len(all_venues_neighborhoods)))

103
1349


In [9]:
# 3. Converting each JSON into a DataFrame and appends it to all_venues_df
all_venues_df = pd.DataFrame()
for result in all_results:
    all_venues_df = all_venues_df.append(json_normalize(venue), sort=False)

# 4. Formatting the DataFrame
all_venues_df["Category"] = all_venues_df["venue.categories"].apply(lambda x: x[0]["name"])
all_venues_df["Neighborhood"] = all_neighborhoods
filtered_columns = ["Neighborhood", "venue.name", "Category", "venue.location.lat", "venue.location.lng"]
all_venues_df = all_venues_df.loc[:, filtered_columns].reset_index(drop=True)
all_venues_df.columns = ["Neighborhood", "Venue", "Venue Category", "Venue Latitude", "Venue Longitude"]

all_venues_df

,Neighborhood,Venue,Venue Category,Venue Latitude,Venue Longitude
0,"Malvern, Rouge",Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,"Malvern, Rouge",Interprovincial Group,Print Shop,43.805630,-79.200378
2,"Rouge Hill, Port Union, Highland Creek",Royal Canadian Legion,Bar,43.782533,-79.163085
3,"Rouge Hill, Port Union, Highland Creek",SEBS Engineering Inc. (Sustainable Energy and ...,Construction & Landscaping,43.782371,-79.156820
4,"Rouge Hill, Port Union, Highland Creek",Scarborough Historical Society,History Museum,43.788755,-79.162438
...,...,...,...,...,...
1344,"South Steeles, Silverstone, Humbergate, Jamest...",McDonald's,Fast Food Restaurant,43.741757,-79.584230
1345,"South Steeles, Silverstone, Humbergate, Jamest...",Tim Hortons,Coffee Shop,43.742015,-79.589690
1346,Northwest,Economy Rent A Car,Rental Car Location,43.708471,-79.589943
1347,Northwest,Logistics Distribution,Bar,43.707554,-79.589252


In [10]:
# 5. Printing the number of unique venue categories in Toronto neighborhoods
print("There are {} unique categories".format(len(all_venues_df["Venue Category"].unique())))

# 6. Returning the top 5 neighborhoods with the most venues
all_venues_df.groupby("Neighborhood")["Venue"].count().reset_index().sort_values(by=["Venue"], ascending=False).reset_index(drop=True).head()

There are 234 unique categories


,Neighborhood,Venue
0,Willowdale,36
1,"Toronto Dominion Centre, Design Exchange",30
2,"St. James Town, Cabbagetown",30
3,Leaside,30
4,"University of Toronto, Harbord",30


----

## Analyzing Each Neighborhood in Toronto

In order to know which venues are the most common in each neighborhood, the categories are converted into dummy variables, and grouped by neigborhood, which results in a count of each category in each neighborhood. After that, these counts are sorted and categorized, so that a top N is created for each neighborhood. These places are used as columns in a pivot table (neighborhood as index, and venue als value), which shows the most common venues for each neighborhood (from left to right).

In [11]:
# 1. Splitting the values in Venue Category into dummy variables
toronto_dummies = pd.get_dummies(
    all_venues_df["Venue Category"],
    prefix = "",
    prefix_sep = ""
)

# 2. Adding neighborhood as first column into dummies DataFrame
toronto_dummies["neighborhood"] = all_venues_df["Neighborhood"]
fixed_columns = [toronto_dummies.columns[-1]] + list(toronto_dummies.columns[:-1])
toronto_dummies = toronto_dummies[fixed_columns]

print(toronto_dummies.shape)
toronto_dummies.head()

(1349, 235)


,neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [168]:
# 3. Grouping rows by each neighborhood
toronto_grouped = toronto_dummies.groupby("neighborhood").sum().reset_index()

# 5. Creating a DataFrame containing the most common venues in each neighborhood
toronto_common_venues = pd.DataFrame()
for hood in toronto_grouped["neighborhood"]:
    temp_df = toronto_grouped[toronto_grouped["neighborhood"] == hood].T.reset_index()
    temp_df.columns = ["Venue", "Count"]
    temp_df = temp_df.loc[1:]
    temp_df = temp_df[temp_df["Count"] > 0]
    temp_df = temp_df.sort_values(by=["Count"], ascending=False).reset_index(drop=True)
    temp_df["Most Common"] = [i+1 for i in range(temp_df.shape[0])]
    temp_df["Neighborhood"] = hood
    temp_df = temp_df[["Neighborhood", "Most Common", "Venue", "Count"]]
    toronto_common_venues = toronto_common_venues.append(temp_df)
toronto_common_venues.reset_index(drop=True)

toronto_common_venues

,Neighborhood,Most Common,Venue,Count
0,Agincourt,1,Breakfast Spot,1
1,Agincourt,2,Clothing Store,1
2,Agincourt,3,Latin American Restaurant,1
3,Agincourt,4,Lounge,1
4,Agincourt,5,Skating Rink,1
...,...,...,...,...
9,Woodbine Heights,10,Video Store,1
0,York Mills West,1,Park,2
1,York Mills West,2,Bank,1
2,York Mills West,3,Bar,1


In [169]:
# 6. Creating a pivot table where each row shows the top 10 most common venues for each neighborhood
toronto_common_venues_pivot = toronto_common_venues.pivot(
    columns = "Most Common",
    index = "Neighborhood",
    values = "Venue"
).fillna("")

toronto_common_venues_pivot

Most Common,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Agincourt,Breakfast Spot,Clothing Store,Latin American Restaurant,Lounge,Skating Rink,,,,,,...,,,,,,,,,,
"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Coffee Shop,Dance Studio,Gym,Pharmacy,Pub,Sandwich Place,Skating Rink,,...,,,,,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Sushi Restaurant,Supermarket,Shopping Mall,Sandwich Place,Restaurant,Pizza Place,Middle Eastern Restaurant,...,,,,,,,,,,
Bayview Village,Bank,Café,Chinese Restaurant,Japanese Restaurant,,,,,,,...,,,,,,,,,,
"Bedford Park, Lawrence Manor East",Italian Restaurant,Sushi Restaurant,Coffee Shop,Sandwich Place,Restaurant,Juice Bar,Pub,Pizza Place,Pharmacy,Liquor Store,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Wexford, Maryvale",Accessories Store,Auto Garage,Bakery,Breakfast Spot,Middle Eastern Restaurant,Sandwich Place,Shopping Mall,Smoke Shop,,,...,,,,,,,,,,
Willowdale,Coffee Shop,Ramen Restaurant,Café,Sandwich Place,Pizza Place,Arts & Crafts Store,Middle Eastern Restaurant,Sushi Restaurant,Steakhouse,Shopping Mall,...,Ice Cream Shop,Home Service,Grocery Store,Fast Food Restaurant,Electronics Store,Discount Store,Butcher,Bubble Tea Shop,Vietnamese Restaurant,
Woburn,Coffee Shop,Convenience Store,Korean Restaurant,,,,,,,,...,,,,,,,,,,


---

## Clustering the Neighborhoods in Toronto

Because of the dummy variables of the venue categories, I am able to apply the K-Means Clustering model to the dataset. For this dataset, I have used 4 clusters. The clusters are shown in the map below.

In [170]:
from sklearn.cluster import KMeans

# 1. Setting the number of clusters
clusters = 4

# 2. Dropping the neighborhood column
toronto_grouped_clustering = toronto_grouped.drop("neighborhood", axis=1)

# 3. Running K-Mean Clustering
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_

array([0, 0, 3, 0, 3, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 1, 3, 0, 0,
       3, 0, 0, 0, 0, 0, 3, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 1, 0,
       0, 0, 3, 0, 0, 0, 1, 3, 0, 1, 1, 3, 3, 0, 2, 0, 0, 1, 1, 0, 0, 0,
       0, 3, 0, 0, 0])

In [171]:
# 4. Adding clustering labels to toronto_common_venues_pivot
toronto_common_venues_pivot.insert(0, "Cluster Labels", kmeans.labels_)

# 5. Merging neighborhoods_df with toronto_common_venues_pivot
toronto_common_venues_df = pd.DataFrame(toronto_common_venues_pivot.to_records())
toronto_merged = neighborhoods_df.join(toronto_common_venues_df.set_index("Neighborhood"), on="Neighborhood").dropna()
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype("int64")

print(toronto_merged.shape[0])
toronto_merged.head()

98


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1,2,3,4,...,21,22,23,24,25,26,27,28,29,30
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Print Shop,,,...,,,,,,,,,,
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Bar,Construction & Landscaping,History Museum,,...,,,,,,,,,,
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Bank,Breakfast Spot,Electronics Store,Intersection,...,,,,,,,,,,
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Convenience Store,Korean Restaurant,,...,,,,,,,,,,
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,...,,,,,,,,,,


In [172]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# 6. Finding the coordinates of Toronto, Canada
address = "Toronto, CA"
geolocator = Nominatim(user_agent="toronto_finder")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# 7. Creating a map of Toronto
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# 8. Creating colors for each cluster
x = np.arange(clusters)
y = [i+x+(i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(y)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# 9. Adding markers to the map
latitude_values = toronto_merged["Latitude"].values
longitude_values = toronto_merged["Longitude"].values
neighborhood_values = toronto_merged["Neighborhood"].values
cluster_values = toronto_merged["Cluster Labels"].values

for lat, lng, hood, cluster in zip(latitude_values, longitude_values, neighborhood_values, cluster_values):
    label = folium.Popup("{} Cluster {}".format(hood, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7
    ).add_to(map_clusters)

map_clusters

---

## Analyzing Clusters

In order to analyze these four clusters, I have created a function that returns (1) the number of neighborhoods in each cluster, and (2) the top 10 most common venues in each cluster. This data give us an idea of how these clusters differ from each other. 

It turns out that the first cluster contain neighborhoods that mainly contain a park and places to sport. The second cluster contains neighborhoods that contain exactly one coffee shop and one cafe, and lots of vegetarian restaurants. The third cluster contains only one neighborhood that has a wide variation of nearby venues, including a pub and a spa. The fourth cluster contains neighborhoods with a wide variation of restaurants.

In [173]:
def get_top_5_venues(dataframe, cluster):
    temp_columns = ["Neighborhood", "Cluster Labels"] + [i for i in dataframe.columns[6:]]
    temp_df1 = dataframe[dataframe["Cluster Labels"] == cluster-1][temp_columns]
    temp_array = temp_df1.values[:,2:]
    temp_list = []
    for neighborhood in temp_array:
        for venue in neighborhood:
            if not venue == "":
                temp_list.append(venue)
    temp_df2 = pd.DataFrame({
        "id": [i for i in range(len(temp_list))],
        "venue": temp_list
    })
    temp_df2 = temp_df2.groupby("venue").id.count().reset_index()
    temp_df2 = temp_df2.sort_values(by="id", ascending=False).reset_index(drop=True)
    temp_df2 = temp_df2.rename(columns={
        "id": "Count",
        "venue": "Venue"
    })
    if temp_df1.shape[0] == 1:
        print("This cluster counts 1 neighborhood.")
    else:
        print("This cluster counts {} neighborhoods.".format(temp_df1.shape[0]))
    return temp_df2.head(10)

### Cluster 1

In [174]:
# 1. Checking cluster 1
get_top_5_venues(toronto_merged, 1)

This cluster counts 67 neighborhoods.


,Venue,Count
0,Park,27
1,Coffee Shop,20
2,Pizza Place,16
3,Bank,13
4,Fast Food Restaurant,12
5,Sandwich Place,12
6,Grocery Store,11
7,Athletics & Sports,10
8,Café,9
9,Liquor Store,9


### Cluster 2

In [175]:
# 1. Checking cluster 2
get_top_5_venues(toronto_merged, 2)

This cluster counts 12 neighborhoods.


,Venue,Count
0,Café,12
1,Coffee Shop,12
2,Restaurant,9
3,Bakery,9
4,Vegetarian / Vegan Restaurant,7
5,Art Gallery,6
6,Hotel,6
7,Italian Restaurant,6
8,Japanese Restaurant,6
9,American Restaurant,6


### Cluster 3

In [176]:
# 1. Checking cluster 3
get_top_5_venues(toronto_merged, 3)

This cluster counts 1 neighborhood.


,Venue,Count
0,Bakery,1
1,Grocery Store,1
2,Trail,1
3,Spa,1
4,Restaurant,1
5,Pub,1
6,Pizza Place,1
7,Juice Bar,1
8,Italian Restaurant,1
9,Ice Cream Shop,1


### Cluster 4

In [177]:
# 1. Checking cluster 4
get_top_5_venues(toronto_merged, 4)

This cluster counts 18 neighborhoods.


,Venue,Count
0,Coffee Shop,18
1,Restaurant,15
2,Café,14
3,Sandwich Place,12
4,Sushi Restaurant,10
5,Pizza Place,9
6,Bank,9
7,Italian Restaurant,8
8,Juice Bar,8
9,Diner,8


---